In [1]:
import numpy as np
import re
import nltk
import pandas as pd
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch_geometric.data import HeteroData
import torch
import torch_geometric.transforms as T
from sklearn.preprocessing import RobustScaler
import pickle
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [1]:
# Read CSV file
import pandas as pd
file_path = r""
df = pd.read_csv(file_path)


In [2]:
df.columns

Index(['Unnamed: 0', 'id', 'text', 'time', 'location', 'followers', 'user_id',
       'favorite_count', 'retweet_count', 'verified', 'rumour', 'reply_id',
       'reply_text', 'reply_time', 'reply_location', 'reply_followers',
       'reply_user_id', 'reply_verified'],
      dtype='object')

In [3]:
df.time = pd.to_datetime(df.time, format='%a %b %d %H:%M:%S +0000 %Y')
df.reply_time = pd.to_datetime(df.reply_time, format='%a %b %d %H:%M:%S +0000 %Y')

### Time for replies and Number of replies

In [4]:
 df['time_diff']=(df.reply_time - df.time).dt.total_seconds()/60

In [5]:
df['reply_number'] = df.groupby('id')['time_diff'].rank(method='dense')

#### Number of replies x Retweet counts

In [6]:
df_posts = df[['id','text','followers','favorite_count','retweet_count','verified',\
  'rumour','user_id']].drop_duplicates().merge(df.groupby(['id']).agg(replies=(\
  'time_diff','count'),first_time_diff=('time_diff','first')).reset_index(),\
  on="id",how="left")

#### Word embeddings

In [7]:
# Download NLTK stopwords
nltk.download('stopwords')

english_stopwords = nltk.corpus.stopwords.words('english')

# function for cleaning data
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt


def clean_text(
    string: str,
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=english_stopwords) -> str:
    """
    A method to clean text
    """
    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # Removing the punctuations
    for x in string.lower():
        if x in punctuations:
            string = string.replace(x, "")

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df_posts['clean_text'] = np.vectorize(remove_pattern)(df_posts['text'], "@[\w]*")
df_posts['clean_text'] = df_posts['clean_text'].str.replace("[^a-zA-Z#]", " ").apply(clean_text)

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_11084/1199893929.py:1: SyntaxWarning: invalid escape sequence '\w'
  df_posts['clean_text'] = np.vectorize(remove_pattern)(df_posts['text'], "@[\w]*")


In [9]:
def embedding_vocab(filepath, word_index,embedding_dim):
    vocab_size = len(word_index) + 1


    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix_vocab

In [10]:
from tensorflow import keras

#from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df_posts['clean_text'])

2024-12-11 01:25:50.985424: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 01:25:51.214832: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-11 01:25:51.374849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733880351.680828   11084 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733880351.769698   11084 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 01:25:52.410593: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [14]:
#import zipfile

#with zipfile.ZipFile('glove.6B.zip', 'r') as zip_ref:
#    zip_ref.extractall('.')

In [11]:
embedding_dim = 100
embedding_matrix_vocab = embedding_vocab(
    'glove.6B.100d.txt', tokenizer.word_index,
embedding_dim)

In [12]:
# Tokenize text
# create the dictionary
sequences = tokenizer.texts_to_sequences(df_posts['clean_text'])

# Padding sequences if necessary
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_sequence_length)

# Function to get embeddings for a sequence
def get_embeddings(sequence, embedding_matrix):
    embeddings = []
    for idx in sequence:
        embeddings.append(embedding_matrix[idx])
    return np.array(embeddings)

# Apply function to get embeddings for each sequence
df_posts['embeddings'] = [get_embeddings(seq, embedding_matrix_vocab) for seq in sequences]

In [13]:
array_avg = []
for i in df_posts.embeddings:
  array_avg.append(np.mean(i,axis=0))
df_posts['embeddings_avg'] = array_avg

#### Reply embeddings

In [14]:
df['clean_reply_text'] = np.vectorize(remove_pattern)(df['reply_text'], "@[\w]*")
df['clean_reply_text'] = df['clean_reply_text'].str.replace("[^a-zA-Z#]", " ").apply(clean_text)

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_11084/2579928606.py:1: SyntaxWarning: invalid escape sequence '\w'
  df['clean_reply_text'] = np.vectorize(remove_pattern)(df['reply_text'], "@[\w]*")


In [15]:
# create the dictionary
tokenizer =  keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(df['clean_reply_text'])

In [16]:
embedding_dim = 100
embedding_matrix_vocab = embedding_vocab(
    'glove.6B.100d.txt', tokenizer.word_index,
embedding_dim)

In [17]:
# Tokenize text
sequences = tokenizer.texts_to_sequences(df['clean_reply_text'])

# Padding sequences if necessary
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_sequence_length)

# Apply function to get embeddings for each sequence
df['reply_embeddings'] = [get_embeddings(seq, embedding_matrix_vocab) for seq in sequences]

In [18]:
array_avg = []
for i in df.reply_embeddings:
  array_avg.append(np.mean(i,axis=0))
df['reply_embeddings_avg'] = array_avg

/home/codespace/miniconda3/envs/mastering/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/codespace/miniconda3/envs/mastering/lib/python3.12/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [19]:

df['reply_embeddings_avg'] = df['reply_embeddings_avg'].apply(lambda x: x if isinstance(x, np.ndarray) else np.zeros(100))

In [20]:
df = df.drop(columns=['Unnamed: 0'])

#### Saving cleaned csv's

In [21]:
df_posts

,id,text,followers,favorite_count,retweet_count,verified,rumour,user_id,replies,first_time_diff,clean_text,embeddings,embeddings_avg
0,498254340310966273,Michael Brown is the 17 yr old boy who was sho...,1034,54,293,False,1,552283163,8,25.733333,michael brown 17 yr old boy shot 10x amp kille...,"[[0.296970009803772, 0.13338999450206757, 0.19...","[0.029599157014959736, -0.1258555807565388, 0...."
1,498254929942028288,17 year old unarmed kid shot ten times by poli...,874288,50,176,False,1,7214972,13,13.383333,17 year old unarmed kid shot ten times police ...,"[[0.44488000869750977, -0.06544700264930725, -...","[0.19377462948775953, 0.10071799257356259, 0.1..."
2,498272309535191041,200 cops in riot gear in #ferguson because the...,605,148,486,False,1,15890403,8,4.766667,200 cops riot gear ferguson prayer circle conc...,"[[0.5302799940109253, 0.8232499957084656, -0.0...","[0.1510988038033247, 0.2517209962010384, -0.08..."
3,498280126254428160,Mike Brown was staying with his grandmother fo...,133561,46,146,True,1,24165761,4,3.150000,mike brown staying grandmother summer lived co...,"[[-0.1477999985218048, 0.3142299950122833, 0.3...","[0.11098087765276432, 0.10974890080979094, 0.0..."
4,498293668655423488,Every 28 hours a black male is killed in the U...,133545,428,1755,True,1,24165761,34,0.366667,every 28 hours black male killed united states...,"[[-0.27726998925209045, 0.926609992980957, 0.1...","[0.1422940082848072, 0.13133273070508783, 0.15..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,500427783848615936,A mother lost her child. A father lost his son...,834,285,448,False,0,27376958,18,3.650000,mother lost child father lost son young man lo...,"[[0.6058700084686279, 0.02798900008201599, 0.0...","[0.4580300028125445, 0.21627932911117873, 0.34..."
1006,500429525558505473,Jesse Jackson leading a small group in prayer ...,7036,115,156,True,0,14715424,25,2.833333,jesse jackson leading small group prayer scene...,"[[-0.039987001568078995, 0.3184700012207031, -...","[0.049042703583836555, 0.1608500976115465, -0...."
1007,500430445830365184,There is a dangerous socioecon war occurring i...,13275,88,152,False,0,1594679162,12,7.766667,dangerous socioecon war occurring us must use ...,"[[-0.09617099910974503, -0.3849399983882904, 0...","[-0.05983392263834293, 0.2960853817371222, 0.2..."
1008,500430673099104256,Wake up. Whenever a black man is killed by pol...,172285,190,124,True,0,19222164,15,0.700000,wake whenever black man killed police try make...,"[[-0.08185099810361862, -0.19044999778270721, ...","[0.021059471865495047, 0.02342099944750468, 0...."


In [22]:
df.to_pickle('replies_ferguson.pkl')

In [23]:
df_posts.to_pickle("posts_ferguson.pkl")

#### Creat Graph

In [147]:
df_posts = pd.read_pickle(r"/workspaces/rumour-detection-pheme/posts_charlie_hebdo.pkl")
df_replies = pd.read_pickle(r"/workspaces/rumour-detection-pheme/replies_charlie_hebdo.pkl")

#### Torch Data

In [149]:
# One-hot encoding
df_posts['verified'] = df_posts['verified'].astype('str').str.\
             replace(' ', '').replace('True', '1').replace('False', '0')\
             .astype('int64')

df_posts = pd.concat([df_posts, pd.get_dummies(\
                          df_posts["verified"],dtype=int)], axis=1, join='inner')
df_posts.drop(["verified"], axis=1, inplace=True)
df_posts.rename(columns={1:'verified',0:'no_verified'},inplace=True)

In [150]:
# One-hot encoding
df_replies['reply_verified'] = df_replies['reply_verified'].astype('str').str.\
             replace(' ', '').replace('True', '1').replace('False', '0')\
             .astype('int64')

df_replies = pd.concat([df_replies, pd.get_dummies(\
                          df_replies["reply_verified"],dtype=int)], axis=1, join='inner')
df_replies.drop(["reply_verified"], axis=1, inplace=True)
df_replies.rename(columns={1:'reply_verified',0:'reply_no_verified'},inplace=True)

In [151]:

# Get unique values from the column
unique_values = df_posts['id'].unique()

# Create a dictionary to map values to IDs
value_to_id = {value: i for i, value in enumerate(unique_values)}

# Map the values to IDs using the dictionary
#retweets_node_features['index'] = retweets_node_features['reply_user_id'].map(value_to_id)
post_map  = value_to_id

#Only keep features
post_features = df_posts[["followers", "favorite_count","retweet_count","no_verified","verified",\
                          "first_time_diff"]]


In [152]:
embeddings_avg = []
for embeddings in df_posts['embeddings_avg']:
  #embeddings= i.replace('[','').replace(']','').replace('\n','').split(' ')
  #embeddings =  [float(item) for item in embeddings if item != '']
  embeddings_avg.append(embeddings)

In [153]:


# Initialize the Robust Scaler
scaler = RobustScaler()

# Assuming data is a DataFrame containing your dataset
scaled_features = scaler.fit_transform(post_features[['followers', 'favorite_count', 'retweet_count', 'first_time_diff']])

# Convert the scaled features back to a DataFrame
scaled_data = pd.DataFrame(scaled_features, columns=['followers', 'favorite_count', 'retweet_count', 'first_time_diff'])

# Add the binary features back to the scaled data
scaled_data['no_verified'] = post_features['no_verified']
scaled_data['verified'] = post_features['verified']
post_features = scaled_data

In [154]:
# Convert to numpy
x1 = np.concatenate((post_features.to_numpy(),np.array(embeddings_avg)),axis=1)
#x1 = post_features.to_numpy()
x1.shape # [num_movie_nodes x movie_node_feature_dim]

(2002, 106)

In [155]:
# Select node features
retweets_node_features = df_replies[["reply_followers", "reply_no_verified", "reply_verified","reply_user_id","time_diff"]]

# Get unique values from the column
unique_values = retweets_node_features['reply_user_id'].unique()

# Create a dictionary to map values to IDs
value_to_id = {value: i for i, value in enumerate(unique_values)}

# Map the values to IDs using the dictionary
#retweets_node_features['index'] = retweets_node_features['reply_user_id'].map(value_to_id)
retweets_id_mapping  = value_to_id




#Only keep features
retweets_node_features = retweets_node_features[["reply_followers", "reply_no_verified", "reply_verified","time_diff"]]


In [156]:
embeddings_avg = []
for embeddings in df_replies['reply_embeddings_avg']:
  #embeddings= i.replace('[','').replace(']','').replace('\n','').split(' ')
  #embeddings =  [float(item) for item in embeddings if item != '']
  embeddings_avg.append(embeddings)

In [157]:
# Initialize the Robust Scaler
scaler = RobustScaler()

# Assuming data is a DataFrame containing your dataset
scaled_features = scaler.fit_transform(retweets_node_features[['reply_followers','time_diff']])

# Convert the scaled features back to a DataFrame
scaled_data = pd.DataFrame(scaled_features, columns=['reply_followers','time_diff'])

# Add the binary features back to the scaled data
scaled_data['reply_no_verified'] = retweets_node_features['reply_no_verified']
scaled_data['reply_verified'] = retweets_node_features['reply_verified']
retweets_node_features = scaled_data


In [158]:
# Convert to numpy
#x2 = np.concatenate((retweets_node_features.to_numpy(),np.array(embeddings_avg)),axis=1)
x2 = retweets_node_features.to_numpy()
x2.shape # [num_movie_nodes x movie_node_feature_dim]

(19050, 4)

In [159]:
# Extract labels
labels = df_posts.rumour
y = labels.to_numpy()
y.shape

(2002,)

In [160]:
# Map post IDs
#post_map = posts_id_mapping.reset_index().set_index("id").to_dict()
df_replies["id"] = df_replies['id'].map(post_map).astype(int)
# Map user IDs
#retweets_map = retweets_id_mapping #retweets_id_mapping.reset_index().set_index("reply_user_id").to_dict()
df_replies["reply_user_id"] = df_replies["reply_user_id"].map(retweets_id_mapping)#.astype(int)

In [161]:
edge_index = df_replies[["id", "reply_user_id"]].values.transpose()
edge_index # [2 x num_edges]

array([[    0,     0,     0, ...,  2001,  2001,  2001],
       [    0,     1,     2, ...,   284,  1810, 14464]])

In [162]:

# Number of rows
num_rows = x1.shape[0]

# Desired proportions
train_proportion = 0.70
val_proportion = 0.15
test_proportion = 0.15

# Generate a list of indices and shuffle them
indices = np.arange(num_rows)
np.random.shuffle(indices)

# Calculate the split indices
train_end = int(train_proportion * num_rows)
val_end = train_end + int(val_proportion * num_rows)

# Split the indices into train, validation, and test
train_indices = indices[:train_end]
val_indices = indices[train_end:val_end]
test_indices = indices[val_end:]

# Create masks
train_mask = np.zeros(num_rows, dtype=bool)
val_mask = np.zeros(num_rows, dtype=bool)
test_mask = np.zeros(num_rows, dtype=bool)

# Assign True to the corresponding indices
train_mask[train_indices] = True
val_mask[val_indices] = True
test_mask[test_indices] = True


In [163]:
# Initialize the HeteroData object
data = HeteroData()

# Add node features and labels for the 'paper' node type
data['id'].x = torch.tensor(x1,dtype=torch.float32)  # Node features dtype=torch.float32
data['id'].y = torch.from_numpy(y)  # Node labels (for example, binary classification)
data['id'].train_mask = torch.tensor(train_mask)  # Training mask
data['id'].val_mask =torch.tensor(val_mask)  # Validation mask
data['id'].test_mask = torch.tensor(test_mask)  # Test mask

# Add node features for the 'author' node type
data['reply_user_id'].x = torch.tensor(x2,dtype=torch.float32) #torch.float32

# Add edges for the (id, retweet, reply_D) relation
data['id', 'retweet', 'reply_user_id'].edge_index = torch.from_numpy(edge_index.reshape(2,len(x2)))
data = T.ToUndirected()(data)
print(data)


HeteroData(
  id={
    x=[2002, 106],
    y=[2002],
    train_mask=[2002],
    val_mask=[2002],
    test_mask=[2002],
  },
  reply_user_id={ x=[19050, 4] },
  (id, retweet, reply_user_id)={ edge_index=[2, 19050] },
  (reply_user_id, rev_retweet, id)={ edge_index=[2, 19050] }
)


In [164]:
print("train:",data['id'].y[train_mask].sum(),data['id'].y[train_mask].shape)

train: tensor(307) torch.Size([1401])


In [165]:
print("val:", data['id'].y[val_mask].sum(),data['id'].y[val_mask].shape)

val: tensor(65) torch.Size([300])


In [166]:
print("test:", data['id'].y[test_mask].sum(),data['id'].y[test_mask].shape)

test: tensor(75) torch.Size([301])


In [167]:
with open('charlie_hebdo_graph_dataset_node_embeddings_v2.pkl', 'wb') as f:
    pickle.dump(data, f)

In [168]:

import torch_geometric.transforms as T
from sklearn.metrics import precision_score, recall_score

In [169]:
# Load the HeteroData object from the pkl file
with open('charlie_hebdo_graph_dataset_node_embeddings_v2.pkl', 'rb') as f:
    data = pickle.load(f)